In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder

NameError: name '_C' is not defined

In [ ]:
# Choosing gpu or cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Data Preprocessing

In [ ]:
# Defining the path to train and test data folders
train_folder = 'Data/train'
test_folder = 'Data/test'

# Image transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.RandomResizedCrop(200),
    transforms.RandomHorizontalFlip(),
    transforms.GaussianBlur((5,9),(0.1,5)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the train and test datasets
train_dataset = ImageFolder(train_folder, transform=transform)
test_dataset = ImageFolder(test_folder, transform=transform)

print(f"Images in training data : {len(train_dataset)}")
print(f"Images in test data : {len(test_dataset)}")

Preparing Dataset

In [ ]:
# Checking the size/structure of tensors we have created
img, label = train_dataset[0]
print(img.shape, label)

In [ ]:
# Checking for the number of classes
print("The following classes are there : \n",train_dataset.classes)

In [ ]:
# Splitting the train dataset into train and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders for train, validation, and test datasets
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Model Building

Defining the CNN Architecture: 4 convolutional layers and an outer layer.

1.   Convolutional Layer Structure: Is a Convolutional Layer (Input image, Filter) with a ReLU activation function and a MaxPooling Layer
2.   Fully Connected Layer: Has a Linear Activation function that takes on an Input vector of size N (size of the resized images) and gives an output of size K = 2 (2 classes  tomatoes and apples)
3.  Forward propagation function
4. Training and Validation steps method implementation


In [ ]:
# Define the CNN model architecture
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

        )
        self.fc_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 6 * 6, 512), #size of input has to match nbr columns in matrix 1 (batch size, input size)
            nn.ReLU(),
            nn.Linear(512,64),
            nn.ReLU(),
            nn.Linear(64, 2),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)
        return x

    def training_step(self, batch):
        images, labels = batch[0].to(device), batch[1].to(device)
        out = self(images) # Generate predictions
        loss = nn.functional.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch[0].to(device), batch[1].to(device)
        out = self(images) # Generate predictions
        loss = nn.functional.cross_entropy(out, labels) # Calculate loss
        acc = accuracy(out, labels) # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}

    @staticmethod
    def validation_epoch_end(outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean() # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean() # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

    @staticmethod
    def epoch_end(epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

Hyper-parameter tuning

In [ ]:
# Defining the accuracy, evaluation and fit methods
def accuracy(outputs, labels):
  _, preds = torch.max(outputs, dim=1)
  return torch.tensor(torch.sum(preds == labels).item() / len(preds))

@torch.no_grad()
def evaluate(model, val_loader):
  model.eval()
  outputs = [model.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)

    return history

In [ ]:
# model summary
model = CNNModel().to(device)
model

Model Training

In [ ]:
# Train the model
num_epochs = 50
lr =0.001
opt_func = optim.Adam

history = fit(num_epochs, lr, model, train_loader, val_loader, opt_func)

Model Evaluation

In [ ]:
def plot_accuracies(history):
    print("Plot the history of accuracies")
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs')

plot_accuracies(history)

In [ ]:
def plot_losses(history):
    print("Plot the losses in each epoch")
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')

plot_losses(history)

In [ ]:
# Evaluate model on test data
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {(100 * correct / total):.2f}%")